# Cross Tab
From the questions in Chapter 8:
> Write the code to create a crosstab representation of the MovieLens data (you might need to do some web searching!).

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 5.5MB/s 
     |████████████████████████████████| 194kB 8.8MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 1.2MB 11.7MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *

In [3]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [9]:
import numpy as np

In [4]:
path

Path('/root/.fastai/data/ml-100k')

In [5]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
len(ratings['user'].unique())

943

In [7]:
len(ratings['movie'].unique())

1682

In principle it is OK to loop through all the users and through all the movies and save the scores of the (`user`, `movie`) pairs:

In [14]:
ratings_small = ratings.iloc[:10]

In [15]:
def makeCrossTab(df, rowsName, columnsName, valueName):
  data = {}
  for u in df[rowsName].unique():
    data[u] = {}
    for m in df[columnsName].unique():
      score = np.nan
      if len(df[ (df[rowsName]==u) & (df[columnsName]==m) ]) > 0:
        score = df[ (df[rowsName]==u) & (df[columnsName]==m) ][valueName].values[0]
      data[u][m] = score
  return data

In [16]:
data = makeCrossTab(ratings_small, 'user', 'movie', 'rating')

In [19]:
data.keys()

dict_keys([196, 186, 22, 244, 166, 298, 115, 253, 305, 6])

In [21]:
pd.DataFrame(data).T

,242,302,377,51,346,474,265,465,451,86
196,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
166,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
298,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
115,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [23]:
ratings[ (ratings['user']==196) & (ratings['movie']==302) ]['rating']

Series([], Name: rating, dtype: int64)

This of course is super inefficient, and I had to run it on a small subset of data to get it to work in a reasonable amount of time :\

It turns out `pandas` has a method for that, of course :)

In [13]:
pd.crosstab(ratings['user'], ratings['movie'], values=ratings['rating'], aggfunc=np.mean)

movie  1     2     3     4     5     6     7     8     9     10    11    12    \
user                                                                            
1       5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0   2.0   5.0   
2       4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0   NaN   NaN   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   4.0   NaN   
5       4.0   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
939     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   5.0   NaN   NaN   NaN   
940     NaN   NaN   NaN   2.0   NaN   NaN   4.0   5.0   3.0   NaN   NaN   4.0   
941     5.0   NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   
942     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
943     NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN   3.0   NaN   4.0   5.0   

movie  13    14    15    16    17    18    19    20    21    22    23    24    \
user                                                                            
1       5.0   5.0   5.0   5.0   3.0   4.0   5.0   4.0   1.0   4.0   4.0   3.0   
2       4.0   4.0   NaN   NaN   NaN   NaN   3.0   NaN   NaN   NaN   NaN   NaN   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5       NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN   3.0   NaN   NaN   4.0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
939     NaN   NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
940     NaN   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
941     NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
942     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
943     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   4.0   4.0   4.0   

movie  25    26    27    28    29    30    31    32    33    34    35    36    \
user                                                                            
1       4.0   3.0   2.0   4.0   1.0   3.0   3.0   5.0   4.0   2.0   1.0   2.0   
2       4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5       3.0   NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
939     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
940     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
941     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
942     NaN   NaN   NaN   NaN   NaN   NaN   5.0   NaN   NaN   NaN   NaN   NaN   
943     NaN   NaN   4.0   4.0   NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   

movie  37    38    39    40    41    42    43    44    45    46    47    48    \
user                                                                            
1       2.0   3.0   4.0   3.0   2.0   5.0   4.0   5.0   5.0   4.0   4.0   5.0   
2       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5       NaN   NaN   NaN   4.0   NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
939     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
940     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   3.0   NaN   
941     NaN   NaN   NaN   Na